In [1]:
import pandas as pd
import requests
import json
import numpy as np
import praw
import time

# Setting  up

In [2]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('9wXh5oa4cfW07_eUn5Hu3A', 'm3GnhaEvbM5LGCWX3BMghLCatOLN3g')
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'babi_research/0.0.1'}
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'softyarn',
        'password': '1532679!Qa1',
        'User-Agent':'babi_research/0.0.1',
        'refresh_token': '167791397396-1cjKrPIOvr4ct1EFwMm57hY1vi7baw'
        }

In [3]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res.json()

{'access_token': '167791397396-Bwssruw6BrT33PCRaLmRRQVm9LO4gw',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [4]:
# convert response to JSON and pull access_token value
token = res.json()['access_token']
# add authorization to our headers dictionary
headers['Authorization'] = 'bearer {}'.format(token)
# while the token is valid (~2 hours) we just add headers=headers to our requests. End point: GET /api/v1/me
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': False,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_softyarn',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_3ef3ey/styles/profileIcon_snoo86ebddf0-4659-47c8-9ad6-b6ca0588943d-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=86680ec95737b7ce10db712e2889672a4de8bbf1',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size

# Miscellanous scraping

In [6]:
#Get moderators of feminisms
fems= requests.get('https://oauth.reddit.com/r/feminisms/about/moderators', headers=headers).json()

In [7]:
print(fems)

{'kind': 'UserList', 'data': {'children': [{'name': 'gleemie', 'author_flair_text': None, 'mod_permissions': ['all'], 'date': 1204540797.0, 'rel_id': 'rb_47ks', 'id': 't2_jk2i', 'author_flair_css_class': None}, {'name': 'alvaspiral', 'author_flair_text': None, 'mod_permissions': ['all'], 'date': 1262761591.0, 'rel_id': 'rb_45ryx', 'id': 't2_3rf60', 'author_flair_css_class': None}, {'name': 'yellowmix', 'author_flair_text': None, 'mod_permissions': ['all'], 'date': 1262815441.0, 'rel_id': 'rb_45ymi', 'id': 't2_2hqa5', 'author_flair_css_class': None}, {'name': 'reddit_feminist', 'author_flair_text': None, 'mod_permissions': ['all'], 'date': 1311325826.0, 'rel_id': 'rb_i6yqm', 'id': 't2_5d186', 'author_flair_css_class': None}, {'name': 'Saferbot', 'author_flair_text': None, 'mod_permissions': ['all'], 'date': 1332000605.0, 'rel_id': 'rb_11zr6f', 'id': 't2_6ladm', 'author_flair_css_class': None}, {'name': 'AutoModerator', 'author_flair_text': None, 'mod_permissions': ['all'], 'date': 16088

# Get comments

## Write function

In [8]:
def clean_data(source_dir, comment_bool):
    df = pd.read_csv(source_dir)
    df = df[["id","body"]]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    df["note"] = np.where(df["body"] == "[removed]", "removed", np.where(df["body"] == "[deleted]", "deleted", "text"))
    return df

In [52]:
fds[0:1]

,id,body,note
0,t1_ex0vz70,"Men hit the wall too by the way. We get fat, ...",text


In [74]:
def get_comments(df) :
    tic = time.time()
    global res_df
    res_df = []
    i = 0
    while i < len(df["id"]):
        j = min(len(df["id"]), i + 40)
        x = ",".join(df["id"][i:j])
        comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'id': x}).json()
        for k in range(j-i):
            comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(comment_df)
        i += 40
        if i%300 == 0:
            time.sleep(2)
            #now = time.localtime(time.time())
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)
            #print(time.strftime("Time now: %HH:%M:%S",now),f', i = {i}')
    return pd.concat(res_df)

In [ ]:
def get_submissions(df):
    res_df = pd.DataFrame()
    i = 0
    while i < len(df["id"]):
        x = ",".join(df["id"][i:i+20])
        submissions = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': x}).json()
        for k in range(20):
            submission_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df = pd.concat([res_df,submission_df])
        if i%300 == 0:
            time.sleep(5)
        i += 20  
    return res_df  

## Scraping

In [12]:
fds = clean_data("E:/gihub-data/redditbots/fds/fds_comments.csv", comment_bool= True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_29520\2879732105.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_dir)


In [78]:
fds_res = get_comments(fds)

Time elapsed:  8  secs, i =  600
Time elapsed:  15  secs, i =  1200
Time elapsed:  23  secs, i =  1800
Time elapsed:  31  secs, i =  2400
Time elapsed:  39  secs, i =  3000
Time elapsed:  47  secs, i =  3600
Time elapsed:  55  secs, i =  4200
Time elapsed:  63  secs, i =  4800
Time elapsed:  70  secs, i =  5400
Time elapsed:  78  secs, i =  6000
Time elapsed:  86  secs, i =  6600
Time elapsed:  94  secs, i =  7200
Time elapsed:  101  secs, i =  7800
Time elapsed:  109  secs, i =  8400
Time elapsed:  116  secs, i =  9000
Time elapsed:  123  secs, i =  9600
Time elapsed:  131  secs, i =  10200
Time elapsed:  138  secs, i =  10800
Time elapsed:  146  secs, i =  11400
Time elapsed:  154  secs, i =  12000
Time elapsed:  162  secs, i =  12600
Time elapsed:  170  secs, i =  13200
Time elapsed:  177  secs, i =  13800
Time elapsed:  185  secs, i =  14400
Time elapsed:  192  secs, i =  15000
Time elapsed:  200  secs, i =  15600
Time elapsed:  208  secs, i =  16200
Time elapsed:  215  secs, i =  

In [79]:
fds_res.to_csv('fds_comment_retrieved.csv')